In [28]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import glm

from sklearn.metrics import mean_absolute_error,mean_squared_error, accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#### correlated feature removed 

In [5]:
df=pd.read_csv(r'C:\Melbourne_Escooter\RealDataset\FinalResults\SA1\Features-trips\Final\AllFeatures-original - wholeNumberDensity - removed_female_50-64_40-49_house - Copy.csv')

In [6]:
column=['tripDensity']
df_depe=df.drop(column, axis=1)
df_depe

,SA1_CODE21,date,hour,trainDensity,busDensity,tramDensity,Cafe perc,Office perc,Shops perc,entropy,...,recreationCount,campusCount,5-14 %,15-29 %,30-39 %,above65 %,male%,Population density 2021,rainfall,min_temp
0,20604111702,1,0,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,1.352866,...,0,0,3.536138,53.399850,19.330746,8.158310,49.511571,9456.2,0,7.4
1,20604111702,1,1,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,1.352866,...,0,0,3.536138,53.399850,19.330746,8.158310,49.511571,9456.2,0,7.4
2,20604111702,1,2,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,1.352866,...,0,0,3.536138,53.399850,19.330746,8.158310,49.511571,9456.2,0,7.4
3,20604111702,1,3,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,1.352866,...,0,0,3.536138,53.399850,19.330746,8.158310,49.511571,9456.2,0,7.4
4,20604111702,1,4,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,1.352866,...,0,0,3.536138,53.399850,19.330746,8.158310,49.511571,9456.2,0,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56611,20604151016,7,19,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,1.118100,...,0,0,1.813717,45.837792,34.053583,4.732242,51.647198,9238.5,1,8.0
56612,20604151016,7,20,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,1.118100,...,0,0,1.813717,45.837792,34.053583,4.732242,51.647198,9238.5,1,8.0
56613,20604151016,7,21,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,1.118100,...,0,0,1.813717,45.837792,34.053583,4.732242,51.647198,9238.5,1,8.0
56614,20604151016,7,22,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,1.118100,...,0,0,1.813717,45.837792,34.053583,4.732242,51.647198,9238.5,1,8.0


In [7]:
df_depe=df_depe.drop(['SA1_CODE21'], axis=1)

In [8]:
df_depe["hour"].replace({0:1,1:1,2:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4,11:4,12:5,13:5,14:5,15:6,16:6,17:6,18:7,19:7,20:7,21:8,22:8,23:8}, inplace=True)
df_depe["date"].replace({1:1,2: 1, 3: 1,4: 1,5: 1,6: 2,7: 2}, inplace=True)
df_depe['trainDensity'] = df['trainDensity'].map(lambda x: 1 if x>0 else 0)

In [9]:
categorical =['hour','date','rainfall','trainDensity']
for col in categorical:
    df_depe[col]=df_depe[col].astype('category')

In [10]:
df_depe=pd.get_dummies(df_depe, drop_first=True)
df_depe.columns

Index(['busDensity', 'tramDensity', 'Cafe perc', 'Office perc', 'Shops perc',
       'entropy', 'mxi', 'recreationCount', 'campusCount', '5-14 %', '15-29 %',
       '30-39 %', 'above65 %', 'male%', 'Population density 2021', 'min_temp',
       'date_2', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'trainDensity_1', 'rainfall_1'],
      dtype='object')

In [11]:
scale= StandardScaler()
df_sc= scale.fit_transform(df_depe)
df_sc=pd.DataFrame(df_sc, columns=df_depe.columns)

In [12]:
df['tripDensity'] = df['tripDensity'].map(lambda x: 1 if x>49.0 else 0)
df['tripDensity']=df['tripDensity'].astype('category')

In [13]:
y=df['tripDensity']
X=df_sc

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1,random_state=42,max_iter=1000)

In [16]:
model = clf.fit(X_train, y_train)

In [17]:
ypredict = clf.predict(X_test)

In [37]:
print("MAE %f" %mean_absolute_error(y_test, ypredict))


MAE 0.189597


In [36]:
print("MSE  %f" %mean_squared_error(y_test, ypredict))

MSE  0.189597


In [22]:
import numpy as np
print("RMSE  %f" %np.sqrt(mean_squared_error(y_test, ypredict)))

RMSE  0.435428


In [116]:
metrics.f1_score(y_test, ypredict)

0.29768392370572205

In [117]:
y_pred_proba = clf.predict_proba(X_test)[::,1]
auc = metrics.roc_auc_score(y_test, y_pred_proba)
print(auc)

0.7893516893546624


In [118]:
import statsmodels.api as sm
X_train =sm.add_constant(X_train)

In [119]:
model=sm.Logit(y_train,X_train)

In [120]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.408887
         Iterations 7


In [121]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            tripDensity   No. Observations:                45292
Model:                          Logit   Df Residuals:                    45265
Method:                           MLE   Df Model:                           26
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:                  0.1807
Time:                        11:02:42   Log-Likelihood:                -18519.
converged:                       True   LL-Null:                       -22604.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -1.8680      0.018   -105.849      0.000      -1.903      -1.833
busDensity                 -0.0644      0.014     -4.502      0.000      -0.092      -0.036
tramDensity                 0.1728      0.015     11.365      0.000       0.143       0.203
Cafe perc                   0.3401      0.016     21.006      0.000       0.308       0.372
Office perc                 0.1624      0.011     14.145      0.000       0.140       0.185
Shops perc                  0.1019      0.011      8.903      0.000       0.079       0.124
entropy                    -0.0890      0.024     -3.653      0.000      -0.137      -0.041
mxi                         0.0970      0.028      3.517      0.000       0.043       0.151
recreationCount            -0.1794      0.029     -6.260      0.000      -0.236      -0.123
campusCount                -0.0271      0.013     -2.015      0.044      -0.053      -0.001
5-14 %                     -0.5335      0.036    -14.892      0.000      -0.604      -0.463
15-29 %                    -0.6888      0.073     -9.432      0.000      -0.832      -0.546
30-39 %                    -0.8215      0.050    -16.481      0.000      -0.919      -0.724
above65 %                  -0.8831      0.085    -10.379      0.000      -1.050      -0.716
male%                       0.6799      0.047     14.610      0.000       0.589       0.771
Population density 2021     0.1609      0.027      5.856      0.000       0.107       0.215
min_temp                    0.1858      0.018     10.210      0.000       0.150       0.221
date_2                      0.3908      0.020     19.977      0.000       0.352       0.429
hour_2                     -0.2983      0.020    -14.658      0.000      -0.338      -0.258
hour_3                     -0.1731      0.019     -9.173      0.000      -0.210      -0.136
hour_4                      0.0089      0.017      0.510      0.610      -0.025       0.043
hour_5                      0.1240      0.017      7.399      0.000       0.091       0.157
hour_6                      0.2296      0.016     13.947      0.000       0.197       0.262
hour_7                      0.1828      0.017     11.030      0.000       0.150       0.215
hour_8                      0.1816      0.017     10.947      0.000       0.149       0.214
trainDensity_1             -0.0428      0.015     -2.844      0.004      -0.072      -0.013
rainfall_1                 -0.1328      0.015     -8.621      0.000      -0.163      -0.103
===========================================================================================
"""

In [29]:
# cross validation

In [31]:
test_score= cross_val_score(model, X_test, y_test, scoring='neg_mean_squared_error')

In [35]:
np.mean(np.absolute(test_score))

0.18933252209455614